In [1]:
import pandas as pd
import random
import numpy as np
import random
import tensorflow as tf
import operator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv("../HW-2-Machine-reasoning/dataset1.csv")
data.head(10)

,x,f(x)
0,-0.66,18.37
1,2.99,5.00
2,2.65,5.12
3,-1.72,27.31
4,4.33,6.76
5,1.94,6.13
6,-1.02,21.17
7,0.05,13.73
8,-1.77,27.78
9,1.06,8.76


In [3]:
shuffle(data)

,x,f(x)
4944,0.72,10.19
13087,0.86,9.57
10597,-4.11,55.59
20107,-1.37,24.07
15097,-2.35,33.59
...,...,...
3833,3.30,5.09
2261,-4.10,55.46
19825,2.82,5.03
10331,0.58,10.88


In [5]:
x_train, x_test, y_train, y_test = train_test_split(data['x'],data['f(x)'], train_size=0.80, random_state=0)

In [6]:
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [7]:
F = ['+','-','*','/']

In [8]:
T = ['x']+ list(range(-5,6))

In [9]:
ran = [F,T]

In [10]:
tf = [True,False]

In [11]:
print(T)

['x', -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]


In [12]:
class Node():
    def __init__(self, *args):
        if(len(args) > 1):
            self.val = args[0]
            self.left = args[1]
            self.right = args[2]
        else:
            self.val = args[0]
            self.left = None
            self.right = None
            
    def insert_left(self, child):
        if (self.left == None):
            self.left = child
        else:
            child.left = self.left
            self.left = child
    def insert_right(self, child):
        if(self.right == None):
            self.right = child
        else:
            child.right = self.right
            self.right = child
    def evaluate(self, x):
        if(self.val in T):
            if(self.val == 'x'):
                return int(x)
            else:
                return int(self.val)
        elif(self.val in F):
            if(self.val == '+'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left + right
            elif(self.val == '-'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left - right
            elif(self.val == '*'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left * right
            elif(self.val == '/'):
                if(self.right.evaluate(x) == 0):
                    return 1
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left / right
    def fitness(self, x, y):
        return self.evaluate(x)-y
#         return abs(self.evaluate(x)-y)

        

In [13]:

def print2D(root, space) :

    if (root == None) :
        return
    # Increase distance between levels
    space += 10

    print2D(root.right, space)

    # Print current node after space
    print()
    for i in range(10, space):
        print(end = " ")
    print(root.val)

    print2D(root.left, space)



In [14]:
def calculate_mse(x_data, y_data, tree):
    total = 0
    for i in range(len(x_data)):
        total += (tree.fitness(x_data[i],y_data[i]) ** 2)
    return total/len(x_data)

In [15]:
def generate_tree_shapes(size):
    root = Node(random.choice(random.choice(ran)))
    for i in range(size):
        if(random.choice(tf)):
            root.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.right.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.right.insert_left(Node(random.choice(random.choice(ran))))
        
        if(random.choice(tf)):
            root.insert_left(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.left.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.left.insert_left(Node(random.choice(random.choice(ran))))

    return root

In [16]:

def generate_tree(size, root):
    if(size == 1):
        root.right = Node(random.choice(T))
        root.left = Node(random.choice(T))
    else:
        #right node
        newNode = random.choice(ran)
        if (newNode == F):
            root.right = Node(random.choice(F))
            generate_tree(size-1, root.right)
        else:
            root.right = Node(random.choice(T))
        
        #left node
        newNode = random.choice(ran)
        if (newNode == F):
            root.left = Node(random.choice(F))
            generate_tree(size-1,root.left)
        else:
            root.left = Node(random.choice(T))
root = Node(random.choice(F))

In [17]:
def mutateHelper():
    mutation = Node(random.choice(F))
    (generate_tree(3, mutation))
    return mutation

def mutate(tree):
    path = []
    path = randomSubTree(tree, path)
    mutation = mutateHelper()
    tree = replaceNode(tree, path, mutation)
    return tree


rl = [-1,1]
tf = [True, True, True, False]
def randomSubTree(node, path):
    keepGoing = random.choice(tf)
    if(keepGoing and len(path) < 3):
        nextNode = random.choice([node.right, node.left])
        if(nextNode == None):
            return path
        if(nextNode == node.right):
            path.append(1)
            randomSubTree(node.right, path)
        else:
            path.append(-1)
            randomSubTree(node.left, path)
    return path

# this function returns a subtree given a specific path
def getNode(node, path):
    if(path == []):
        return node
    if(path[0]==1):
        return getNode(node.right, path[1::])
    else:
        return getNode(node.left, path[1::])

#this function changes a node in one tree to another given node
def replaceNode(nodeA, pathA, nodeB):
    if(pathA == []):
        return nodeB
    elif(pathA[0] == 1):
        return Node(nodeA.val, nodeA.left, replaceNode(nodeA.right, pathA[1::], nodeB))
    elif(pathA[0] == -1):
        return Node(nodeA.val, replaceNode(nodeA.left, pathA[1::], nodeB), nodeA.right)

        

def crossover(treeA, treeB): 
    # determine a random node in each for crossover
    pathA = []
    pathB = []
    pathA = randomSubTree(treeA, pathA)
    pathB = randomSubTree(treeB, pathB)
    # swapping the random node in both trees
    temp = getNode(treeA, pathA)
    treeA = replaceNode(treeA, pathA, getNode(treeB, pathB))
    treeB = replaceNode(treeB, pathB, temp)
    return treeA, treeB
    
            
    

In [18]:
def tournament_selection(population, k):
    parents = random.choices(population, k = 5)
    best = 99999
    better = 99999
    best_parent = None
    better_parent = None
    for i in parents:
        mse = calculate_mse(x_train, y_train, i)
        if (mse < best):
            best = mse
            best_parent = i
    for i in parents:
        mse = calculate_mse(x_train, y_train, i)
        if (mse < better and i != best_parent):
            better = mse
            better_parent = i
    if(k==1):
        return best_parent
    return best_parent, better_parent

In [73]:
# population = []
# lowest_pop = None
# lowest_val = 10000000

# for i in range(10):
#     tree = Node(random.choice(F))
#     generate_tree(10, tree)
#     population.append(tree)

# epochs = 200

# for i in range(epochs):
#     new_generation = []
#     w = []
#     j = 0
#     for i in population:
#         w.append(calculate_mse(x_train, y_train, i))
#     total = sum(w)
# #     print(w,min(w))
#     for i in range(len(w)):
#         w[i] = (total-w[i])/total
#     while(j < 10):
#         operator = random.choices(['r','m','c'], weights = [2,5,6], k = 1)[0]
#         if(operator == 'r'):
#             child = random.choices(population, weights = w, k = 1)[0]
#             new_generation.append(child)
#             j+=1
#         elif(operator == 'm'):
#             child = random.choices(population, weights = w, k = 1)[0]
#             child = mutate(child)
#             new_generation.append(child)
#             j+=1
#         elif(operator == 'c'):
#             children = random.choices(population, weights = w, k = 2)
#             childA = children[0]
#             childB = children[1]
#             crossover(childA, childB)
#             new_generation.append(childA)
#             new_generation.append(childB)
#             j+=2
#     population = []
#     for i in new_generation:
#         population.append(i)
#     for i in range(len(population)):
#         calc = calculate_mse(x_test,y_test,population[i])
#         print("Population " +str(i) + ": " +str(calc))
#         if calc < lowest_val:
#             lowest_val = calc
#             lowest_pop = population[i]
#     print(" ")

25.86

In [59]:
len(population)

10

In [72]:
for i in range(len(population)):
    print("Population " +str(i) + ": " +str(calculate_mse(x_test,y_test,population[i])))

Population 0: 1259.8856261822327
Population 1: 1341.7751692933234
Population 2: 891.3755575600119
Population 3: 1156.3817793674007
Population 4: 1539.0266412933408
Population 5: 919.5789135609436
Population 6: 1539.0266412933408
Population 7: 919.5789135609436
Population 8: 919.5789135609436
Population 9: 877.3075271037501
Population 10: 877.3075271037501


In [71]:
lowest_val

159.67935062666706

In [26]:
def find_pop(epochs = 200):
    population = []
    population_mse = []
    lowest_pop = None
    lowest_val = 10000000

    for i in range(10):
        tree = Node(random.choice(F))
        generate_tree(10, tree)
        population.append(tree)

    epochs = epochs

    for i in range(epochs):
        print("Epoch: " + str(i) + "/" + str(epochs))
        new_generation = []
        w = []
        j = 0
        best = 99999
        
        pop_mse = 99999
        pop_low = None
        
        for i in population:
            best =  min(best, calculate_mse(x_test,y_test,i))
        
        print("Test: ", best)

        
        while(j < 10):
            operator = random.choices(['r','m','c'], weights = [3,3,4], k = 1)[0]
            if(operator == 'r'):
                child = tournament_selection(population, 1)
                new_generation.append(child)
                j+=1
            elif(operator == 'm'):
                child = tournament_selection(population, 1)
                child = mutate(child)
                new_generation.append(child)
                j+=1
            elif(operator == 'c'):
                childA,childB = tournament_selection(population, 2)
                childA,childB = crossover(childA, childB)
                new_generation.append(childA)
                new_generation.append(childB)
                j+=2        

        for i in new_generation:
    #         population.append(i)
            calc = calculate_mse(x_train,y_train,i)
            if calc < pop_mse:
                pop_low = i
                pop_mse = calc
        pop_high = None
        pop_high_mse = 0
        pop_high_index = None

        for i in range(len(population)):
            calc = calculate_mse(x_train,y_train,population[i])
            population_mse.append(calc)
            if calc > pop_high_mse:
                pop_high_mse = calc
                pop_high = population[i]
                pop_high_index = i
        population[pop_high_index] = pop_low
        population_mse[pop_high_index] = pop_mse
        print("Lowest Newly Generated MSE: "+ str(pop_mse))
        print("Lowest MSE Overall: " + str(min(population_mse)))
        print("Replaced MSE: " + str(pop_high_mse))

        for i in range(len(population)):
            calc = calculate_mse(x_train,y_train,population[i])
    #         print("Population " +str(i) + ": " +str(calc))
            if calc < lowest_val:
                lowest_val = calc
                lowest_pop = population[i]
        print(" ")
    return lowest_pop,population

In [ ]:
find_pop(50)

Epoch: 0/50
Test:  621.606934719999
Lowest Newly Generated MSE: 535.5996521099996
Lowest MSE Overall: 535.5996521099996
Replaced MSE: 5067.458228110027
 
Epoch: 1/50
Test:  532.6262387200001
Lowest Newly Generated MSE: 535.5996521099996
Lowest MSE Overall: 535.5996521099996
Replaced MSE: 2623.611869396689
 
Epoch: 2/50
Test:  532.6262387200001
Lowest Newly Generated MSE: 535.5996521099996
Lowest MSE Overall: 535.5996521099996
Replaced MSE: 2497.326442721126
 
Epoch: 3/50
Test:  532.6262387200001
Lowest Newly Generated MSE: 437.7251811099995
Lowest MSE Overall: 437.7251811099995
Replaced MSE: 1351.8282113377643
 
Epoch: 4/50
Test:  435.554044719998
Lowest Newly Generated MSE: 371.0560851099999
Lowest MSE Overall: 371.0560851099999
Replaced MSE: 860.4099003743497
 
Epoch: 5/50
Test:  368.9641487200001
Lowest Newly Generated MSE: 371.0560851099999
Lowest MSE Overall: 371.0560851099999
Replaced MSE: 857.2005401099957
 
Epoch: 6/50
Test:  368.9641487200001
Lowest Newly Generated MSE: 371.05

In [ ]:
print("Best performing on train set: " +str(calculate_mse(x_test,y_test,low)))

In [ ]:
for i in range(len(pops)):
    print("Population " +str(i) + ": " +str(calculate_mse(x_test,y_test,pops[i])))